# **Azure AI Agents with Knowledge Tools (Bing Search)**

## Overview
This notebook demonstrates how to create and use AI agents with knowledge tools in Azure AI Foundry. You'll learn how to enhance an AI agent's capabilities by integrating external knowledge sources like Bing search, allowing it to provide up-to-date information in response to user queries.

### Stateless vs. Stateful APIs: Understanding the Difference

#### **Completions API (Stateless)**
In the previous notebooks, we used the OpenAI Completions API, which is **stateless**:
- Each request is independent and does not retain memory of previous interactions.
- Conversation history must be manually included in every request.
- Simple request-response pattern with no built-in conversation management.

```python
# Stateless Completions API example
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me about Mars."}
    ]
)
```


#### **Agent API (Stateful) workflow**
- Maintains conversation context through threads and runs.
- Automatically manages conversation history and state.
- Enables persistence of interactions for multi-turn conversations.

**1. Create a thread (conversation container)**
```python
thread = project_client.agents.create_thread()
```

**2. Add messages to the thread**
```python
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Tell me about Mars."
)
```

**3. Run the agent on the thread**
```python
python
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, 
    agent_id=agent.id
)
```

## Learning Objectives
- Set up an Azure AI agent with access to Bing search capabilities
- Configure system instructions for knowledge-grounded responses
- Create and manage conversation threads
- Send queries that require current information
- Process responses that incorporate real-time search results
- Implement proper citation formats for information sources

## Prerequisites
- An Azure account with access to Azure AI Foundry
- A Bing connection configured in your Azure AI Foundry project
- Appropriate environment variables in a `.env` file:
  - `PROJECT_CONNECTION_STRING`: Connection string for your Azure AI Foundry project
  - `BING_CONNECTION_NAME`: Name of your Bing connection
  - `chatModel`: The model to use for the agent (e.g., GPT-4)

## Workflow
This notebook walks through the complete lifecycle of an AI agent with knowledge tools:
1. Setting up the environment and client
2. Creating a Bing knowledge tool
3. Defining instructions for how the agent should use search capabilities
4. Creating the agent with the knowledge tool
5. Managing conversation threads and messages
6. Running the agent to process queries
7. Cleaning up resources when finished

Follow along step by step to create your own knowledge-enhanced AI agent!

**Imports**

In [14]:
import os
import dotenv
dotenv.load_dotenv(".env")

True

**Create Client**

In [15]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

**Knowledge Tools**

In [16]:
from azure.ai.projects.models import BingGroundingTool

bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

**Create Agent**

In [17]:
system_message=(
    "You are a knowledgeable assistant with access to Bing search capabilities."
    "\n When answering questions:"
    "\n     1. For factual queries, use the Bing tool to search for up-to-date information."
    "\n     2. Always cite your sources when providing information from Bing."
    "\n     3. If the search results are insufficient, acknowledge limitations and offer what you know."
    "\n     4. Summarize complex information in a clear, concise manner."
    "\n     5. When appropriate, organize information with bullet points or numbered lists."
    "\n     6. Avoid making claims that aren't supported by your search results."
    "\n     7. For time-sensitive information, note when the data was retrieved."
    "\n     8. If a query is ambiguous, ask clarifying questions before searching."
    "\n     9. If the user asks for a specific format (e.g., table), provide it if possible."
    "\n     10. For citations, please use the IEEE (Institute of Electrical and Electronics Engineers) format."
    "\n           - How you should apply it:"
    "\n                 a. For in-text citations, use numbered citations in brackets [1]."
    "\n                 b. At the end of the report, you should provide a list of citations in the format "
    "(the list should ONLY contain the sources used in the free text of the research report. "
    "Do NOT list sources which are not cited in the free text of the research report.):"
    "\n                     [1] Title of the source, URL."
    "\n                 c. The list should be numbered in the order they appear in the text."
    "\n                 d. If the same source/url is cited multiple times, it should be listed only once in the citation list."
    "\n                 e. Pay extra attention in the end so that you do not create multiple citations lists."
    "\n\nYour goal is to provide accurate, helpful responses while clearly distinguishing between search results and your own knowledge."
)

# print(system_message)

In [18]:
from azure.ai.projects.models import FunctionTool

agent = project_client.agents.create_agent(
    name="my-knowledge-tool-agent",
    model=os.getenv("chatModel"),
    instructions=system_message,
    tools=bing.definitions
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_1LT0foxE0bJMj37hFxVt2lR1



**Create Thread**

In [19]:
thread = project_client.agents.create_thread()
agent = project_client.agents.get_agent(agent_id=agent.id)

**Adding first message**

In [20]:
from azure.ai.projects.models import MessageRole

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What is the latest news in AI when it comes to OpenAI?",
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_oxm8cvwuj6qhRO2y9NRG5Ayz



**Running Agent with Thread**

In [21]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: completed



In [22]:
from utils.helper import pretty_print_thread_messages

messages = project_client.agents.list_messages(thread_id=thread.id)
pretty_print_thread_messages(messages)


👤 USER
------------------------------------------------------------


What is the latest news in AI when it comes to OpenAI?


🤖 ASSISTANT
------------------------------------------------------------


OpenAI has made several important announcements recently:                                                          

 1 Fundraising Efforts: OpenAI is set to raise up to $40 billion in a new funding round led by SoftBank Group, with
   a valuation of $300 billion. This funding is aimed at advancing AI research, expanding computational            
   infrastructure, and enhancing its tools[4][3].                                                                  
 2 Launch of Open-Weight Model: OpenAI plans to launch its first open-weight language model since GPT-2, aiming to 
   provide more accessibility to its AI models[2].                                                                 
 3 Image Generation Tool: The latest image generation tool by ChatGPT has seen significant engagement, with users  
   generating 700 million images in a few days. This new tool also sparked a trend of AI-generated images in the   
   style of Studio Ghibli[1].                                                                                      

For more details, you can visit the cited sources.


Citations:
1. ChatGPT’s image surge: 700 million generated in days as India leads AI boom: https://www.fortuneindia.com/technology/chatgpts-image-surge-700-million-generated-in-days-as-india-leads-ai-boom/121603
2. OpenAI is planning on launching its first open-weight model in years: https://www.msn.com/en-us/news/technology/openai-is-planning-on-launching-its-first-open-weight-model-in-years/ar-AA1C4Hrg
3. OpenAI plans to raise $40 billion to boost its AI efforts: https://edition.cnn.com/2025/04/01/business/openai-funding-round/index.html
4. OpenAI to raise $40 billion in SoftBank-led round to boost AI efforts: https://www.msn.com/en-us/technology/artificial-intelligence/openai-to-raise-40-billion-in-softbank-led-round-to-boost-ai-efforts/ar-AA1C576K


**Adding second message to thread**

In [23]:
from azure.ai.projects.models import MessageRole

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Which industries are most affected?",
)

**Running Agent with Thread**

In [24]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: completed



In [25]:
from utils.helper import pretty_print_thread_messages

messages = project_client.agents.list_messages(thread_id=thread.id)
pretty_print_thread_messages(messages)


👤 USER
------------------------------------------------------------


What is the latest news in AI when it comes to OpenAI?


🤖 ASSISTANT
------------------------------------------------------------


OpenAI has made several important announcements recently:                                                          

 1 Fundraising Efforts: OpenAI is set to raise up to $40 billion in a new funding round led by SoftBank Group, with
   a valuation of $300 billion. This funding is aimed at advancing AI research, expanding computational            
   infrastructure, and enhancing its tools[4][3].                                                                  
 2 Launch of Open-Weight Model: OpenAI plans to launch its first open-weight language model since GPT-2, aiming to 
   provide more accessibility to its AI models[2].                                                                 
 3 Image Generation Tool: The latest image generation tool by ChatGPT has seen significant engagement, with users  
   generating 700 million images in a few days. This new tool also sparked a trend of AI-generated images in the   
   style of Studio Ghibli[1].                                                                                      

For more details, you can visit the cited sources.


Citations:
1. ChatGPT’s image surge: 700 million generated in days as India leads AI boom: https://www.fortuneindia.com/technology/chatgpts-image-surge-700-million-generated-in-days-as-india-leads-ai-boom/121603
2. OpenAI is planning on launching its first open-weight model in years: https://www.msn.com/en-us/news/technology/openai-is-planning-on-launching-its-first-open-weight-model-in-years/ar-AA1C4Hrg
3. OpenAI plans to raise $40 billion to boost its AI efforts: https://edition.cnn.com/2025/04/01/business/openai-funding-round/index.html
4. OpenAI to raise $40 billion in SoftBank-led round to boost AI efforts: https://www.msn.com/en-us/technology/artificial-intelligence/openai-to-raise-40-billion-in-softbank-led-round-to-boost-ai-efforts/ar-AA1C576K

👤 USER
------------------------------------------------------------


Which industries are most affected?


🤖 ASSISTANT
------------------------------------------------------------


OpenAI’s recent developments are affecting several key industries:                                                 

 1 Finance: The integration of AI with blockchain technologies is transforming financial services, including       
   decentralized finance (DeFi) systems and predictive market analytics. This fosters innovations such as          
   intelligent digital identities, AI-driven compliance systems, and secure financial transactions[4].             
 2 Healthcare: AI applications in healthcare are leveraging blockchain-secured patient data for personalized       
   medical treatments. This enhances the ability to analyze vast amounts of data for more accurate diagnostics and 
   treatments[3].                                                                                                  
 3 Logistics and Supply Chain: AI and blockchain together are improving transparency, efficiency, and security in  
   logistics and supply chain management. This integration allows better tracking, inventory management, and       
   automated compliance processes[2].                                                                              
 4 Government and Regulatory Compliance: AI technology is being used to ensure regulatory compliance, reduce fraud,
   and handle complex transactions in government and public sector operations, promoting more efficient and        
   transparent systems[1].                                                                                         

These advancements by OpenAI are driving significant changes across various sectors, showcasing the growing        
importance of AI in modern industry solutions.


Citations:
1. OpenAI’s $40 Billion And Circle IPO: AI And Blockchain’s ... - Forbes: https://www.forbes.com/sites/digital-assets/2025/04/03/openais-40b-and-circle-ipo-ai-and-blockchains-economic-revolution/
2. OpenAI’s $40 Billion And Circle IPO: AI And Blockchain’s ... - Forbes: https://www.forbes.com/sites/digital-assets/2025/04/03/openais-40b-and-circle-ipo-ai-and-blockchains-economic-revolution/Citations:
1. OpenAI’s $40 Billion And Circle IPO: AI And Blockchain’s ... - Forbes: https://www.forbes.com/sites/digital-assets/2025/04/03/openais-40b-and-circle-ipo-ai-and-blockchains-economic-revolution/
2. OpenAI’s $40 Billion And Circle IPO: AI And Blockchain’s ... - Forbes: https://www.forbes.com/sites/digital-assets/2025/04/03/openais-40b-and-circle-ipo-ai-and-blockchains-economic-revolution/
3. OpenAI’s $40 Billion And Circle IPO: AI And Blockchain’s ... - Forbes: https://www.forbes.com/sites/digital-assets/2025/04/03/openais-40b-and-circle-ipo-ai-and-blockchains-economic-revolution

**Delete Agent & Thread**

In [26]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# Delete Thread when done
project_client.agents.delete_thread(thread_id=thread.id)
print(f"thread: {thread.id} deleted")

Deleted agent

thread: thread_bRQzMuCPMkrZJyJvso2jA9wV deleted
